In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import numpy as np
import matplotlib as plt
plt.rc('font', family='Malgun Gothic')
pd.set_option('display.max_columns',100) # 칼럼개수 늘리기
pd.set_option('display.max_rows',50) # 칼럼개수 늘리기

In [2]:
# 파일 읽기
sample =pd.read_csv("sample_submission.csv",encoding='cp949') #제출용
profile = pd.read_csv("train_profiles.csv",encoding= 'cp949') #train 나이
test_click = pd.read_csv("test_clickstreams.tab",encoding='cp949',sep = '\t') 
test_search = pd.read_csv("test_searchkeywords.tab",encoding='cp949',sep = '\t')
train_click = pd.read_csv("train_clickstreams.tab",encoding='cp949',sep = '\t')
train_search = pd.read_csv("train_searchkeywords.tab",encoding='cp949',sep = '\t')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# profile GROUP 변수 분리
# **************한 번만 실행해야해요*********************
profile['gender'] = profile.GROUP.apply(lambda x: x[0]) # 성별
profile['age'] = profile.GROUP.apply(lambda x: x[1:]) # 나이
#del profile['GROUP'] # GROUP 제거

In [3]:
#null 값 날리기
train_click = train_click.dropna()
train_search = train_search.dropna() 

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
train_search = pd.read_csv("train_searchkeywords.tab",encoding='cp949',sep = '\t')
train_search['QRY_CNT'][train_search['QRY_CNT'].isna()] # train_search['QRY_CNT'] null값

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


405906    NaN
1073959   NaN
Name: QRY_CNT, dtype: float64

In [13]:
#---------------------필요한가?---------------------------------
train_click = pd.merge(train_click,profile) # train_click + profile
train_search = pd.merge(train_search,profile) # train_search + profile
del train_click['SITE'] #SITE 변수 삭제
#click = pd.merge(train_click,test_click)  # clicks 통합
#search = pd.merge(train_search, test_search) # search 통합
#--------------------------------------------------------------
#pd.pivot_table(train_click, values='SITE_CNT', index='ACT_NM', columns=['gender','age'], 
#               aggfunc=len, fill_value=0) 
#-----------------------------------------------------------------------------

In [4]:
# train_click ['TIME_ID'] 쪼개기
train_click['DATE'] = train_click['TIME_ID'].apply(lambda x : int(str(x)[:8])) # 년월일
#train_click['YEAR'] = train_click['TIME_ID'].apply(lambda x : str(x)[:4]) # 년도
#train_click['MONTH'] = train_click['TIME_ID'].apply(lambda x : str(x)[4:6]) # 월
#train_click['DAY'] = train_click['TIME_ID'].apply(lambda x : str(x)[6:8]) # 일
train_click['HOUR'] = train_click['TIME_ID'].apply(lambda x : int(str(x)[8:])) # 시간
#요일
def fw(x):
    return(("MON","TUE",'WED','THU','FRI','SAT','SUN')[x.weekday()])
train_click['WEEKDAY'] = pd.to_datetime(train_click.TIME_ID.apply(lambda x : int(str(x)[:8])).astype(str)).apply(fw)
train_click['WEEKDAY'] = train_click.WEEKDAY.astype('category').cat.reorder_categories(["MON","TUE",'WED','THU','FRI','SAT','SUN'],ordered=True)

In [4]:
# train_search, test_search 전처리
train_search['QRY_STR']=train_search['QRY_STR']\
.apply(lambda x : x.split('&')[0] if 'acq' in x else x)\
.apply(lambda x : x.split('&')[-1].split('=')[-1] if 'query' in x else x)\
.apply(lambda x : x.replace('+',' '))
test_search['QRY_STR']=test_search['QRY_STR']\
.apply(lambda x : x.split('&')[0] if 'acq' in x else x)\
.apply(lambda x : x.split('&')[-1].split('=')[-1] if 'query' in x else x)\
.apply(lambda x : x.replace('+',' '))
#train_search['CUS_ID'] = train_search['CUS_ID'].apply(lambda x : str(x))
#train_search['CUS_ID'] = test_search['CUS_ID'].apply(lambda x : str(x)) 

In [8]:
# DC DATE
DC = train_click[['CUS_ID','DATE','ST_TIME']].groupby(['CUS_ID','DATE'],as_index=False).sum()
DC2 = DC[['CUS_ID','ST_TIME']].groupby(['CUS_ID']).mean()
DC2=DC2.reset_index()
DC3 = DC[['CUS_ID','ST_TIME']].groupby(['CUS_ID']).std()
DC3 = DC3.reset_index()
DC3.columns=['CUS_ID','DAYCOV2']
DC2=pd.merge(DC2, DC3, on='CUS_ID')
DC2['DAYCOV']=DC2['DAYCOV2']/DC2['ST_TIME']
del DC2['ST_TIME']
del DC2['DAYCOV2']
DC = DC2
DC
#SC2 = SC[['CUS_ID','ST_TIME']].groupby(['CUS_ID']).mean()
#SC2=SC2.reset_index()
#SC3 = SC[['CUS_ID','ST_TIME']].groupby(['CUS_ID']).std()
#SC3=SC3.reset_index()
#SC3.columns=['CUS_ID','SITECOV2']
#SC2=pd.merge(SC2, SC3, on='CUS_ID')
#SC2['SITECOV']=SC2['SITECOV2']/SC2['ST_TIME']
#del SC2['ST_TIME']
#del SC2['SITECOV2']
#SC=SC2
#SC

,CUS_ID,DAYCOV
0,1,1.481130
1,2,0.484465
2,3,0.700117
3,4,1.144454
4,5,1.057735
5,6,1.195519
6,7,1.258600
7,8,1.402674
8,9,0.991156
9,10,0.564534


In [9]:
#discussion변수 추가
#DWELLTIME 총체류시간(ST_TIME의 총합)
DW=train_click[['CUS_ID','ST_TIME']].groupby('CUS_ID',as_index=False).sum()
DW.columns=['CUS_ID','DWELLTIME']
#PAGEVIEWS 총 페이지뷰(SITE_CNT의 총합)
PV=train_click[['CUS_ID','SITE_CNT']].groupby('CUS_ID',as_index=False).sum()
PV.columns=['CUS_ID','PAGEVIEWS']
#VSITE 접속한 서로 다른 사이트(SITE_NM)의 수
VS=train_click[['CUS_ID','SITE_NM']].drop_duplicates().groupby('CUS_ID',as_index=False).count()
VS.columns=['CUS_ID','VSITE']
#COVERAGE 얼마나 다양한 카테고리에 접속했나
CO=train_click[['CUS_ID','BACT_NM']].drop_duplicates().groupby('CUS_ID',as_index=False).count()
CO['BACT_NM']=CO['BACT_NM']/22
CO.columns=['CUS_ID','COVERAGE']
#VDAYS 총 접속일수
VD=train_click[['CUS_ID','TIME_ID']]
VD['TIME_ID']=VD['TIME_ID']//100
VD=VD.drop_duplicates().groupby('CUS_ID',as_index=False).count()
VD.columns=['CUS_ID','VDAY']
#DAYTIME
DT=train_click[['CUS_ID','TIME_ID','ST_TIME']]
DT['TIME_ID']=DT['TIME_ID']//100
DT=DT.groupby(['CUS_ID','TIME_ID'],as_index=False).sum()
DT=DT.groupby('CUS_ID',as_index=False).mean()
DT=DT[['CUS_ID','ST_TIME']]
DT.columns=['CUS_ID','DAYTIME']
#CT
CT=train_click[['CUS_ID','BACT_NM','ST_TIME']].groupby(['CUS_ID','BACT_NM'],as_index=False).sum()
CT2=CT[['CUS_ID','ST_TIME']].groupby('CUS_ID',as_index=False).sum()
CT2.columns=['CUS_ID','SUM_TIME']
CT=pd.merge(CT, CT2, on='CUS_ID')
CT['ST_TIME']=CT['ST_TIME']/CT['SUM_TIME']
CT=pd.pivot_table(CT[['CUS_ID','BACT_NM','ST_TIME']], values='ST_TIME', index='CUS_ID', columns='BACT_NM', aggfunc=sum, fill_value=0)
CT.reset_index()
#DF 요일당 체류시간 비율
DF=train_click[['CUS_ID','WEEKDAY','ST_TIME']].groupby(['CUS_ID','WEEKDAY'],as_index=False).sum()
DF['ST_TIME']=DF['ST_TIME'].fillna(0)
DF2=DF[['CUS_ID','ST_TIME']].groupby('CUS_ID',as_index=False).sum()
DF2.columns=['CUS_ID','SUM_TIME']
DF=pd.merge(DF, DF2, on='CUS_ID')
DF['ST_TIME']=DF['ST_TIME']/DF['SUM_TIME']
DF=pd.pivot_table(DF[['CUS_ID','WEEKDAY','ST_TIME']], values='ST_TIME', index='CUS_ID', columns='WEEKDAY', aggfunc=sum, fill_value=0)
#HF 시간대별(0-5시,6-11시,12-17시,18-23시) 체류시간 비율
HF=train_click[['CUS_ID','HOUR','ST_TIME']].groupby(['CUS_ID','HOUR'],as_index=False).sum()
HF['ST_TIME']=HF['ST_TIME'].fillna(0)
HF['ff'] = HF['HOUR'].apply(lambda x : 'A0_A5' if (x>=0 and x<=5) else ('A6_A11' if (x>5 and x<=11) else ('P12_P17' if (x>11 and x<=17)  else 'P18_P23')))# 시간
HF=HF.groupby(['CUS_ID','ff'],as_index=False).sum()
del HF['HOUR']
HF2=HF[['CUS_ID','ST_TIME']].groupby('CUS_ID',as_index=False).sum()
HF2.columns=['CUS_ID','SUM_TIME']
HF=pd.merge(HF, HF2, on='CUS_ID')
HF['ST_TIME']=HF['ST_TIME']/HF['SUM_TIME']
HF=pd.pivot_table(HF[['CUS_ID','ff','ST_TIME']], values='ST_TIME', index='CUS_ID', columns='ff', aggfunc=sum, fill_value=0)
# SC
SC = train_click[['CUS_ID','BACT_NM','ST_TIME']].groupby(['CUS_ID','BACT_NM'],as_index=False).sum()
SC2 = SC[['CUS_ID','ST_TIME']].groupby(['CUS_ID']).mean()
SC2=SC2.reset_index()
SC3 = SC[['CUS_ID','ST_TIME']].groupby(['CUS_ID']).std()
SC3=SC3.reset_index()
SC3.columns=['CUS_ID','SITECOV2']
SC2=pd.merge(SC2, SC3, on='CUS_ID')
SC2['SITECOV']=SC2['SITECOV2']/SC2['ST_TIME']
del SC2['ST_TIME']
del SC2['SITECOV2']
SC=SC2
# DC 
DC = train_click[['CUS_ID','DATE','ST_TIME']].groupby(['CUS_ID','DATE'],as_index=False).sum()
DC2 = DC[['CUS_ID','ST_TIME']].groupby(['CUS_ID']).mean()
DC2=DC2.reset_index()
DC3 = DC[['CUS_ID','ST_TIME']].groupby(['CUS_ID']).std()
DC3 = DC3.reset_index()
DC3.columns=['CUS_ID','DAYCOV2']
DC2=pd.merge(DC2, DC3, on='CUS_ID')
DC2['DAYCOV']=DC2['DAYCOV2']/DC2['ST_TIME']
del DC2['ST_TIME']

del DC2['DAYCOV2']
DC = DC2

sample_D=pd.merge(DW,PV, on='CUS_ID')
sample_D=pd.merge(sample_D,VS, on='CUS_ID')
sample_D=pd.merge(sample_D,CO, on='CUS_ID')
sample_D=pd.merge(sample_D,SC, on='CUS_ID')
sample_D=pd.merge(sample_D,VD, on='CUS_ID')
sample_D=pd.merge(sample_D,DT, on='CUS_ID')
sample_D=pd.merge(sample_D,DC, on='CUS_ID')
sample_D=pd.merge(sample_D,CT, on='CUS_ID')
sample_D=pd.merge(sample_D,DF, on='CUS_ID')
sample_D=pd.merge(sample_D,HF, on='CUS_ID')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [10]:
# 소분류기준 변수추가 초안
prob_group=pd.pivot_table(train_click, values='SITE_CNT', index='MACT_NM', columns='GROUP', 
               aggfunc=len, fill_value=0)
# 소분류를 기준으로 검색횟수를 피벗테이블로 만듦
prob_group['group_sum']=prob_group['F20-']+prob_group['F30']+prob_group['F40+']+prob_group['M20-']+prob_group['M30']+prob_group['M40+']
# 각각의 소분류에 대한 검색량의 총합
prob_group['F20-']=prob_group['F20-']/prob_group['group_sum']
#각 분류당 F20-의 확률
prob_group['F30']=prob_group['F30']/prob_group['group_sum']
#각 분류당 F30의 확률
prob_group['F40+']=prob_group['F40+']/prob_group['group_sum']
#각 분류당 F40+의 확률
prob_group['M20-']=prob_group['M20-']/prob_group['group_sum']
#각 분류당 M20-의 확률
prob_group['M30']=prob_group['M30']/prob_group['group_sum']
#각 분류당 M30의 확률
prob_group['M40+']=prob_group['M40+']/prob_group['group_sum']
#각 분류당 M40+의 확률
prob_group.columns=['F20','F30','F40','M20','M30','M40','group_sum']
# 열 이름바꾸기(query문을 돌리려면 이렇게 해야해요)
prob_group


,F20,F30,F40,M20,M30,M40,group_sum
MACT_NM,,,,,,,
B2B,0.066422,0.164218,0.083104,0.103765,0.349250,0.233242,6534
IT/통신,0.075354,0.093256,0.072440,0.101998,0.319734,0.337219,2402
NGO,0.116013,0.129085,0.112745,0.037582,0.282680,0.321895,612
PC게임,0.016911,0.093574,0.189402,0.170237,0.199549,0.330327,887
PC방/게임방,0.063545,0.016722,0.043478,0.016722,0.755853,0.103679,299
SaaS/ASP,0.113037,0.180464,0.080812,0.116325,0.341161,0.168201,25850
가격비교,0.100626,0.180745,0.093486,0.056425,0.301657,0.267062,99726
가구/인테리어 쇼핑몰,0.122064,0.249040,0.109627,0.076309,0.286197,0.156763,6513
가전,0.081522,0.153727,0.102484,0.078416,0.248447,0.335404,1288


In [192]:
train_search.iloc[[405907,1073960],:] #더러운데이터 두개

,CUS_ID,QRY_STR,QRY_CNT
405907,"\t4.000000\r\n1732""",ㄷㅁ뇨,1.0
1073960,"\t3.000000\r\n577""",ㅇ큐,1.0


In [5]:
# 더러운 값 찾아내기
def check(x):
    try:
        return int(x)
    except:
        return "X"
        
train_search['CUS_ID'] = train_search['CUS_ID'].apply(lambda x: check(x))
train_search=train_search.query('CUS_ID != "X"')

In [2]:
train_search.head()

,CUS_ID,QRY_STR,QRY_CNT
0,1,못내,1.0
1,1,배트맨 리부트,1.0
2,1,배트맨 비긴즈 명대사,1.0
3,1,베수비오 화산,1.0
4,1,베이징올림픽 장미란,1.0


In [7]:
#-----------------------------------임시------------------------------------
import pandas as pd
train_search = pd.read_csv("train_searchkeywords.tab",encoding='cp949',sep = '\t')
train_search = train_search.dropna() 
# train_search, test_search 전처리
train_search['QRY_STR']=train_search['QRY_STR']\
.apply(lambda x : x.split('&')[0] if 'acq' in x else x)\
.apply(lambda x : x.split('&')[-1].split('=')[-1] if 'query' in x else x)\
.apply(lambda x : x.replace('+',' '))
# 더러운 값 찾아내기
def check(x):
    try:
        return int(x)
    except:
        return "X"
        
train_search['CUS_ID'] = train_search['CUS_ID'].apply(lambda x: check(x))
train_search=train_search.query('CUS_ID != "X"')
#---------------------------------임시----------------------------------------------
df = pd.DataFrame({'CUS_ID':[],'keys':[],'values':[]}) #최종 테이블
df.to_csv('test.csv', index=False) #초기 csv파일 생성

In [10]:
for cus in range(1,2501): #CUS_ID 범위 설정
    temp = {} # 임시 사전 생성
    print(cus,end='\r')
    strList = list(train_search.query('CUS_ID == @cus').QRY_STR.values)
    cntList = list(train_search.query('CUS_ID == @cus').QRY_CNT.values)
    for words , clicks in  zip(strList, cntList):#검색어(word)와 클릭수(cliks)를 하나씩 가져온다
        for key in words.split(' '):# 검색어를 space로 나누어 key를 만든다..
            try:
                temp[key] += clicks # word가 있다면 1을 더하고 --> 클릭횟수를 더해야함
            except:
                temp[key] = clicks #없다면 1을 할당한다.
    #--------------------여기까지 사전완성-----------------------------------
    df = pd.DataFrame({'CUS_ID':cus,'keys':list(temp.keys()),'values':list(temp.values())})
    df.to_csv('test.csv',mode='a', header =False,index=False) #csv파일 덮어씌우기

In [13]:
df=pd.read_csv('test.csv')

In [ ]:
pd.pivot_table(df,index='CUS_ID',columns='keys',values='values',fill_value=0) # keyword 횟수 테이블

In [ ]:
train_click['BACT_NM'].apply(lambda x : str(x)).value_counts().plot.pie()

In [ ]:
# 임용고시가 있는 CUS_ID, GROUP 확인
word = train_click['ACT_NM'].str.contains("임용고시")
pd.DataFrame(train_click.query('@word').groupby(['CUS_ID','GROUP']).count().iloc[:,0]).rename(columns={'TIME_ID':'COUNT'})

In [ ]:
x = train_click[['SITE_CNT','ST_TIME']]
y = train_click['age']

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.24, random_state=0)
print(len(X_train) == len(y_train)
,len(X_test) == len(y_test))

In [ ]:
from sklearn.svm import SVC

svm = SVC(kernel='rbf', C=1.0, random_state=0)
# 로지스틱 회귀에서의 C와 반대의 개념. 모델을 조율해주는 값이라고 보면 됨.
svm.fit(X_train, y_train)
y_pred_svc = svm.predict(X_test)
print('Accuracy: %.2f' % accuracy_score(y_test, y_pred_svc))


np.random.seed(0)
X_xor = np.random.randn(200, 2)
y_xor = np.logical_xor(X_xor[:, 0] > 0,
                       X_xor[:, 1] > 0)
y_xor = np.where(y_xor, 1, -1)

plt.scatter(X_xor[y_xor == 1, 0],
            X_xor[y_xor == 1, 1],
            c='b', marker='x',
            label='1')
plt.scatter(X_xor[y_xor == -1, 0],
            X_xor[y_xor == -1, 1],
            c='r',
            marker='s',
            label='-1')

plt.xlim([-3, 3])
plt.ylim([-3, 3])
plt.legend(loc='best')
plt.tight_layout()
# plt.savefig('./figures/xor.png', dpi=300)
plt.show()


svm = SVC(kernel='rbf', C=10.0, random_state=0, gamma=0.10)
svm.fit(X_xor, y_xor)
y_pred_ksvc = svm.predict(X_xor)
print('Accuracy: %.2f' % accuracy_score(y_xor, y_pred_ksvc))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [ ]:
# SITE_NM 28876가지
# BACT_NM 22가지
# MACT_NM 207가지
# ACT_NM 1216가지
# CUS_ID 2500명
# 2012070100부터 2013063023까지 결과(1년치)
# 단순히 이용량도 변수가 될 수 있을 것
# 대분류별 소분류 찾아보고 이걸로 특정나이대가 들어갈만한것을 임의로 점수를 주고 그 점수가 높으면 나이대 고르는 걸로
# tr_sk영어 들어가면 무조건 쿼리문, 한글만 추려내면 됨